# cell2func

> Convert desired notebook cells to functions. 

Detects function inputs automatically and function outputs semi-automatically. In the latter case, hints are provided to the developer to refine the list of outputs per each cell.

In [ ]:
#| default_exp core.cell2func

In [ ]:
#| export
from dataclasses import dataclass
from functools import reduce
from pathlib import Path
import sys
import ast
from IPython import get_ipython
from IPython.core.magic import (Magics, magics_class, line_magic,
                                cell_magic, line_cell_magic)
import ipynbname
from sklearn.utils import Bunch
from fastcore.all import argnames
import nbdev

In [ ]:
#|export
class FunctionProcessor (Bunch):
    def to_file (self, file_path, mode='w'):
        with open (file_path, mode=mode) as file:
            file.write (self.code)
    
    def write (self, file):
        file.write (self.code)
    
    def update_code (
        self, 
        arguments=None, 
        return_values=None,
        tab_size=4,
        display=True
    ) -> None:
        if arguments is not None:
            self.arguments = arguments
        arguments = ', '.join (self.arguments)
        if return_values is not None:
            self.return_values = return_values
        return_values = ','.join (self.return_values)
        function_code = ''
        for line in self.original_code.splitlines():
            function_code += f'{" " * tab_size}{line}\n'
        if return_values != '':
            return_line = f'return {return_values}'
            return_line = f'{" " * tab_size}{return_line}\n'
        else:
            return_line = ''
        function_code = f'def {self.name}({arguments}):\n' + function_code + return_line
        self.code = function_code
        get_ipython().run_cell(function_code)
        if display:
            print (function_code)

In [ ]:
#| export
class CellProcessor():
    """
    Processes the cell's code according to the magic command.
    """
    def __init__(self, **kwargs):
        self.function_info = Bunch()
        self.function_list = []
        self.file_name = ipynbname.name().replace ('.ipynb', '.py')
        self.nbs_folder = self.get_nbs_path ()
        self.lib_folder = self.get_lib_path ()
        self.file_path = Path(ipynbname.path ().replace(self.nbs_folder, self.lib_folder)) / self.file_name
        
    def cell2file (self, folder, cell):
        folder = Path(folder)
        folder.mkdir(parents=True, exist_ok=True)

        with open(folder / "module.py", "w") as file_handle:
            file_handle.write(cell)

        get_ipython().run_cell(cell)
                    
    def function (
        self,
        func, 
        cell,
        collect_variables_values=True,
        make_function=True,
        tab_size=4,
        update_previous_functions=True,
        discover_outputs=True
    ) -> None:
        
        this_function = FunctionProcessor (
            idx=len(self.function_list), 
            original_code=cell, 
            name=func, 
            discover_outputs=discover_outputs
        )
        if func not in self.function_info:
            self.function_info[func] = this_function
            self.function_list.append (this_function)
            
        idx = this_function.idx
        
        # get variables specific about this function
        if collect_variables_values:
            get_variables_before_code = f'\nkeep_variables ("{func}", "values_before", locals ())'
            get_ipython().run_cell(get_variables_before_code)
            
            get_variables_here_code = cell + f'\nkeep_variables ("{func}", "values_here", locals ())'
            get_ipython().run_cell(get_variables_here_code)
            values_before, values_here = this_function['values_before'], this_function['values_here']
            values_here = {k:values_here[k] for k in set(values_here).difference(values_before)}
            this_function['values_here'] = values_here
            print (values_here)
        
        root = ast.parse (cell)
        variables_here = {node.id for node in ast.walk(root) if isinstance(node, ast.Name) and not callable(eval(node.id))}
        print (variables_here)
        if idx > 0:
            variables_before = reduce (lambda x, y: x['variables_here'] | y['variables_here'], self.function_list[:idx])
        else:
            variables_before = []
        variables_here = sorted (variables_here.difference(variables_before))
        print (variables_here)
        this_function.update (variables_here=variables_here, variables_before=variables_here+variables_before, variables_after=[])
        
        if make_function:
            this_function.update_code ( 
                arguments=variables_before, 
                return_values=variables_here+variables_before,
                tab_size=tab_size
            )
            
        for function in self.function_list[:idx]:
            function.variables_after += [v for v in this_function.variables_here if v not in function.variables_after]
            if update_previous_functions and function.discover_outputs:
                this_function.update_code (function, return_values=function.variables_after, tab_size=tab_size)
    
    def write_functions (self, line):
        with open (str(self.file_path), 'w') as file:
            for function in self.function_list:
                function.write (file)
        
    def get_lib_path (self):
        return nbdev.config.get_config()['lib_path']
                   
    def get_nbs_path (self):
        return nbdev.config.get_config()['nbs_path']

In [ ]:
#| export
@magics_class
class CellProcessorMagic (Magics):
    """
    Base magic class for converting cells to modular functions.
    """
    def __init__(self, shell, **kwargs):
        super().__init__(shell)
        self.processor = CellProcessor (magic=self, **kwargs)
        
    @cell_magic
    def cell2file (self, folder, cell):
        self.processor.cell2file (folder, cell)
    
    @cell_magic
    def function (self, func, cell):
        "Converts cell to function"
        self.processor.function (func, cell)
    
    @line_magic
    def write_functions (self, line):
        return self.write_functions (line)
        
    @line_magic
    def cell_processor (self, line):
        return self.processor

In [ ]:
#| export
#| hide
def load_ipython_extension(ipython):
    """
    This module can be loaded via `%load_ext core.cell2func` or be configured to be autoloaded by IPython at startup time.
    """
    magics = CellProcessorMagic(ipython)
    ipython.register_magics(magics)

In [ ]:
#| export
def keep_variables (function, field, variable_values, self=None):
    """
    Store `variables` in dictionary entry `self.variables_field[function]`
    """
    frame_number = 1
    while not isinstance (self, Cell2Func):
        fr = sys._getframe(frame_number)
        args = argnames(fr, True)
        if len(args)>0:
            self = fr.f_locals[args[0]]
        frame_number += 1
    variable_values = {k: variable_values[k] for k in variable_values if not k.startswith ('_') and not callable(variable_values[k])}
    function_info = getattr(self, 'function_info')
    function_info[function][field]=variable_values

In [ ]:
load_ipython_extension (get_ipython())

In [ ]:
%%function final
a = 1
b = 2
print (f'a + b is {a+b}')
c = a*b
print (f'a * b is {c}')

a + b is 3
a * b is 2
{}
{'a', 'b', 'c'}
['a', 'b', 'c']
def final():
    a = 1
    b = 2
    print (f'a + b is {a+b}')
    c = a*b
    print (f'a * b is {c}')
    return a,b,c



In [ ]:
a = %cell_processor 1

In [ ]:
a.file_name

'cell2func'

In [ ]:
final ()

a + b is 3
a * b is 2


(1, 2, 2)

In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()